In [1]:
import numpy as np
np.random.seed(13)
import pandas as pd
import predictonlyup
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras import optimizers
from __future__ import division
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

def findMaxAcc(valacc,valloss):
	prevloss=valloss[0]
	for i in range(1,len(valloss)):
		if(valloss[i]>prevloss):
			return valacc[i-1]
	#print('Problems with find MaxAccuracy')
	return valacc[len(valacc)-1]

ticker='ADBE'
price= pd.read_csv('/home/andrea/Desktop/NLFF/DataSetIndexesLabeled/indexes'+ticker+'.csv')
if(ticker=='AAPL'):
    ticker='AAPL_tot'
sentimentVector =  pd.read_csv('SentimentNews/'+ticker+'.csv')

#alignment of data
maxdata=max(sentimentVector['initTime'])
mindata=min(sentimentVector['initTime'])
price=price[price['Unnamed: 0']>=mindata]
price=price[price['Unnamed: 0']<=maxdata]
#remove the time from data
sentimentVector=sentimentVector.drop(['Unnamed: 0', 'initTime' ], axis=1)

#2
#label because of the market and append values without datatime
#Important to set lenth of trend in this to see if news affect from today to today+tendwindowtime
#this is theorical (looks at difference betwwen price before news and price after news)because for make it real i have to change price.iloc[i-1]['open'] to price.iloc[i]['open']
#because the trader can buy only on price.iloc[i]['open'] that is today
trendwindowtime=[1,4,12,20,24,26,30,35,40,48,60]
mean=[1,10,20,30]
for m in mean:
    print('000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')
    print('000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

    print('Working on MEAN:',m)
    sentimentVectort1=pd.rolling_mean(sentimentVector,m)
    sentimentVectort=sentimentVectort1.iloc[m:]
    
    pricet=price.iloc[m:]
    for t in trendwindowtime:
        #1
    #label because of the maket and append values without data
    #simo theroy past trend
        x=[]
        y=[]
        print('============================================================')
        print('============================================================')
        print('Working on window:',t)
        #MEAN on the word count
        #print(sentimentVector)



        #print(sentimentVector)

        for i in range(1,len(pricet)-t):
            y.append(np.sign(pricet.iloc[i+t]['close']-pricet.iloc[i-1]['open']))
            x.append(sentimentVectort.iloc[i].values)
        x=np.array(x)
        y=np.array(y)
        y=(y+1)/2

        #Split betwwen train-validation and test
        train=0.8
        nt=math.ceil(len(x)*train)
        x_tv=[]
        y_tv=[]
        x_test=[]
        y_test=[]
        x_tv=x[:nt]
        y_tv=y[:nt]
        x_test=x[nt:]
        y_test=y[nt:]

        #Fairly sampling the test 50% 50%
        posindex=np.where( y_test == 1 )
        negindex=np.where( y_test == 0 )

        yindex=[]
        nindex=min(len(posindex[0]),len(negindex[0]))

        #for i in range(1,nindex):
        y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
        x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))

        nfold=10
        kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


        #model selection
        learningrate=[0.1,0.01,0.001,0.0001,0.00001]
        drop=[0,0.25,0.5]
        bestacc=0
        bestl=0
        bestmodel=None
        cvaccuracy=[]
        print('ModelSelection')
        for l in learningrate:

            #print(l)
            #crossvalidation
            opt=keras.optimizers.SGD(lr=l, momentum=0.0, decay=0.0, nesterov=False)
            model = Sequential()
            model.add(Dense(1, input_shape=(5,), activation='sigmoid'))

            model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
            cvaccuracy=[]
            for train_index, test_index in kf.split(x_tv):
                x_train, x_val = x_tv[train_index], x_tv[test_index]
                y_train, y_val = y_tv[train_index], y_tv[test_index]
                history=model.fit(x_train, y_train, epochs=20, verbose=0,batch_size=10,validation_data=(x_val, y_val))

                maxAcc=findMaxAcc(history.history['val_acc'],history.history['val_loss'])
                cvaccuracy.append(maxAcc)
            mcvaccuracy=sum(cvaccuracy)/len(cvaccuracy)


            if(mcvaccuracy>=bestacc):
                bestacc=mcvaccuracy
                bestl=l
                bestmodel=model

        print('Best Accuracy on validation:',bestacc)
        print('BestLearning:',bestl)
        tot=[]
        print(bestmodel.evaluate(x=x_test, y=y_test, batch_size=10, verbose=0))
        print(bestmodel.evaluate(x=x_testnew, y=y_testnew, batch_size=10, verbose=0))
        print('LOOK HOW IT WORKS WITH TEST SET....')
        opt=keras.optimizers.SGD(lr=bestl, momentum=0.0, decay=0.0, nesterov=False)
        model = Sequential()
        model.add(Dense(1, input_shape=(5,), activation='sigmoid'))

        model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
        model.fit(x_tv, y_tv, epochs=20, verbose=0,batch_size=10)
        evaluation=model.evaluate(x=x_test, y=y_test, batch_size=10, verbose=0)
        evaluation1=model.evaluate(x=x_testnew, y=y_testnew, batch_size=10, verbose=0)
        acc=evaluation[1]
        acc1=evaluation1[1]
        print('Accuracy on test set')
        print(acc)
        print(predictonlyup.alwaysUp(y_test))
        print('on new')
        print(acc1)


/home/andrea/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Working on MEAN: 1
Working on window: 1


/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=1,center=False).mean()


ModelSelection
Best Accuracy on validation: 0.5513910987459641
BestLearning: 0.0001
[0.6997604100766752, 0.4606414035440534]
[0.6998586110126825, 0.45370370517542336]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.492711375692843
0.5247813411078717
on new
0.5216049403412106
Working on window: 4
ModelSelection
Best Accuracy on validation: 0.5324658892486798
BestLearning: 0.01
[0.6913631980182134, 0.5321637457835744]
[0.6917255403740066, 0.526785715483129]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5087719303909798
0.5058479532163743
on new
0.5029761919840461
Working on window: 12
ModelSelection
Best Accuracy on validation: 0.5843548771915275
BestLearning: 0.01
[0.7178037729431107, 0.5073313821445812]
[0.7208800574383104, 0.5000000007181282]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.51319648443429
0.5131964809384164
on new
0.5000000007181282
Working on window: 20
ModelSelection
Best Accuracy on validation: 0.6155882384733566
BestLearning: 0.00

/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=10,center=False).mean()


ModelSelection
Best Accuracy on validation: 0.5226975152978395
BestLearning: 0.1
[0.6970836958927144, 0.46920821503285437]
[0.6975753311044681, 0.46583850811357086]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5219941386991233
0.5249266862170088
on new
0.4968944108634262
Working on window: 4
ModelSelection
Best Accuracy on validation: 0.5432267109922242
BestLearning: 0.0001
[0.7036451627226437, 0.5352941199260599]
[0.705459711063339, 0.5271084339593548]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.45000000210369334
0.5088235294117647
on new
0.46084337645625495
Working on window: 12
ModelSelection
Best Accuracy on validation: 0.5688997850678292
BestLearning: 0.001
[0.6972207898235603, 0.5162241919993651]
[0.7018571666828016, 0.5000000001817215]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5634218349305578
0.5162241887905604
on new
0.5487804903035484
Working on window: 20
ModelSelection
Best Accuracy on validation: 0.61063725834546
BestLearning: 

/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=20,center=False).mean()


ModelSelection
Best Accuracy on validation: 0.5301960829661627
BestLearning: 0.1
[0.7008659275232163, 0.47197640509204525]
[0.7060214757169567, 0.4402515706869791]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5280236018090825
0.528023598820059
on new
0.49999999925025607
Working on window: 4
ModelSelection
Best Accuracy on validation: 0.544891071644224
BestLearning: 0.1
[0.7224688170224252, 0.4112426076503195]
[0.7226838651226788, 0.4207317085891235]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5118343251255842
0.5118343195266272
on new
0.5000000001817215
Working on window: 12
ModelSelection
Best Accuracy on validation: 0.5641348861194962
BestLearning: 0.01
[0.7020992442125382, 0.5192878359060966]
[0.7073012803807671, 0.5000000000919824]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5192878359060966
0.5192878338278932
on new
0.5000000000919824
Working on window: 20
ModelSelection
Best Accuracy on validation: 0.6105583222763127
BestLearning: 0.01


/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=30,center=False).mean()


ModelSelection
Best Accuracy on validation: 0.5503703776884962
BestLearning: 0.1
[0.7068118306225061, 0.4510385772594712]
[0.7114182695557799, 0.4177215190816529]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.45697329522771013
0.5281899109792285
on new
0.42405063192112535
Working on window: 4
ModelSelection
Best Accuracy on validation: 0.560022118472226
BestLearning: 0.1
[0.7218043921249253, 0.46726190489495084]
[0.7257472585748743, 0.4506172839966085]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5148809532235775
0.5148809523809523
on new
0.5000000000919824
Working on window: 12
ModelSelection
Best Accuracy on validation: 0.569723608333673
BestLearning: 0.1
[0.7304510411931507, 0.5194029870317943]
[0.738104060292244, 0.49687499925494194]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5223880605911141
0.5223880597014925
on new
0.5
Working on window: 20
ModelSelection
Best Accuracy on validation: 0.6090113381123905
BestLearning: 1e-05
[0.70243887267